In [0]:
import requests
import json
from requests.auth import HTTPBasicAuth

In [ ]:
# see here for more details on working with the Jira API: 
# https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-issues/#api-group-issues

## Configuration - Store these as Databricks secrets for security
JIRA_URL = "[https://JIRA-HOST-URL.atlassian.net/]"
JIRA_EMAIL = "[JIRA-EMAIL]"
JIRA_API_TOKEN = "[JIRA-API-TOKEN]"  ## Use dbutils.secrets.get() 
PROJECT_KEY = "[JIRA-PROJECT-KEY]"  ## Your Jira project key

In [0]:

def create_jira_ticket(summary, description, issue_type="Task", priority="Medium"):
    """
    Create a Jira ticket via REST API
    
    Args:
        summary (str): Ticket summary/title
        description (str): Ticket description
        issue_type (str): Type of issue (Task, Bug, Story, etc.)
        priority (str): Priority level (Highest, High, Medium, Low, Lowest)
    
    Returns:
        dict: Response containing ticket details or error
    """
    
    url = f"{JIRA_URL}/rest/api/3/issue"
    
    auth = HTTPBasicAuth(JIRA_EMAIL, JIRA_API_TOKEN)
    
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    payload = {
        "fields": {
            "project": {
                "key": PROJECT_KEY
            },
            "summary": summary,
            "description": {
                "type": "doc",
                "version": 1,
                "content": [
                    {
                        "type": "paragraph",
                        "content": [
                            {
                                "type": "text",
                                "text": description
                            }
                        ]
                    }
                ]
            },
            "issuetype": {
                "name": issue_type
            },
            "priority": {
                "name": priority
            }
        }
    }
    
    try:
        response = requests.post(
            url,
            data=json.dumps(payload),
            headers=headers,
            auth=auth
        )
        
        response.raise_for_status()
        
        result = response.json()
        ticket_key = result.get("key")
        ticket_url = f"{JIRA_URL}/browse/{ticket_key}"
        
        print(f"✓ Jira ticket created successfully!")
        print(f"  Ticket Key: {ticket_key}")
        print(f"  URL: {ticket_url}")
        
        return {
            "success": True,
            "ticket_key": ticket_key,
            "ticket_url": ticket_url,
            "response": result
        }
        
    except requests.exceptions.HTTPError as e:
        error_msg = f"HTTP Error: {e.response.status_code} - {e.response.text}"
        print(f"✗ Failed to create Jira ticket: {error_msg}")
        return {
            "success": False,
            "error": error_msg
        }
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(f"✗ Failed to create Jira ticket: {error_msg}")
        return {
            "success": False,
            "error": error_msg
        }

# Example usage in Databricks
if __name__ == "__main__":
    ## In production, use Databricks secrets:
  ## JIRA_API_TOKEN = dbutils.secrets.get(scope="jira", key="api-token")
    ## JIRA_EMAIL = dbutils.secrets.get(scope="jira", key="email")
    
    # Example: Create a ticket for a data pipeline failure
    result = create_jira_ticket(
        summary="Data Pipeline Failed - Daily ETL Job",
        description="The daily ETL job failed at step 3 during data validation. Error: NULL values found in required field 'customer_id'.",
        issue_type="Task",
        priority="High"
    )
    
    # You can use the result in your Databricks workflow
    if result["success"]:
        print(f"Proceed with next steps - Ticket: {result['ticket_key']}")
    else:
        raise Exception(f"Failed to create Jira ticket: {result['error']}")

✓ Jira ticket created successfully!
  Ticket Key: PB-10
  URL: https://databricks-jira-sandbox.atlassian.net//browse/PB-10
Proceed with next steps - Ticket: PB-10
